建立 LLM 会话

In [1]:
from google import genai

with open(r".\api_key", "r", encoding="utf8") as reader: 
    api_key = reader.readline()
client = genai.Client(api_key=api_key)

认证 github

In [2]:
from github.Repository import Repository, NotSet
from github.PullRequest import PullRequest

import github

with open(r".\github_token", "r", encoding="utf8") as reader:
    access_token = reader.readline()

auth = github.Auth.Token(access_token)

筛选 PR

In [3]:
from typing import Literal, List

## https://github.com/{browser-use/browser-use}/pull/xxx.diff

def filtre(
        repo:Repository,
        pr_state:Literal["open", "close", "all"]=NotSet, # pr 状态
        sort:Literal["created", "updated", "pushed", "popularity"]=NotSet, # 结果排序方式
        direction:Literal["asc", "desc"]=NotSet, # 结果顺序
        base:str=NotSet, # 目标分支
        head:str=NotSet, # 来源分支/来源用户
        prid:int=None, # 指定 pr 编号
        author:str=None, # 指定 pr 发起人
    ) -> List[PullRequest]:
    results = repo.get_pulls(pr_state, sort, direction, base, head)
    if prid: results = [pr for pr in results if pr.number == prid]
    if author: results = [pr for pr in results if pr.user.login == author]
    return results

In [4]:
repository = "browser-use/browser-use"

with github.Github(auth=auth) as g:
    repo = g.get_repo(repository)
    filtered = filtre(repo, "open", author="mkuts12")
    if not filtered: raise ValueError("No matching PR found")
    pr = filtered[0]; url = pr.diff_url

In [5]:
# user_prompt = """Please briefly summarize the changes made to the code from the following diff:
# """
user_prompt = """请简短总结以下的 PR 描述和代码差异，并用数字编号输出差异条目：
PR 描述：
{desc}
代码差异：
{diffs}
"""

In [6]:
from utils import file_io_utils as fiu

diffs = fiu.get_remote_contents(url)

response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=[user_prompt.format(desc=f"{pr.title}\n{pr.body}", diffs=diffs)]
)

In [9]:
from IPython.display import Markdown, display

display(Markdown(response.text))

这个 PR 旨在优化点击操作的重试机制，以避免在页面加载超时导致元素消失后，尝试点击不存在元素时仍会失败的问题。之前，如果 `element_handle.click()` 失败（例如，由于页面加载超时），一个通用的 `except` 块会无条件地尝试通过 `page.evaluate()` 进行重试，但这可能在元素确实不存在或页面上下文被销毁时再次失败。

本次变更确保只有当直接的点击操作失败时，才尝试通过 `page.evaluate()` 进行重试，从而更精确地处理点击失败的场景。它不会解决所有情况，特别是当元素被点击但页面已导航导致执行上下文被销毁时。

**代码差异：**

1.  删除了原始的、包裹了 `page.evaluate` 重试逻辑的外部 `except Exception` 块。
2.  引入了一个名为 `_click` 的内部异步辅助函数，封装了点击逻辑。
3.  将使用 `page.evaluate('(el) => el.click()', element_handle)` 的重试逻辑移动到 `_click` 函数内部，具体是在捕获 `element_handle.click(timeout=1500)` 失败的 `except Exception` 块中。这确保了 `page.evaluate` 仅在直接点击失败时才被调用。
4.  最外层的 `perform_click` 现在调用这个新的 `_click` 辅助函数。
5.  简化了外部的 `except Exception` 块，它现在主要捕获从 `_click` 辅助函数或其 `perform_click` 包装器中传播的任何异常。